In [1]:
!python -V

Python 3.9.19


In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [24]:
import mlflow

mlflow.set_tracking_uri("sqlite:///../mlruns.db")
mlflow.set_experiment("nyc-green-taxi-trip-explore")

<Experiment: artifact_location='/workspaces/MLOpsExplore/Notebook/mlruns/2', creation_time=1722177205773, experiment_id='2', last_update_time=1722177205773, lifecycle_stage='active', name='nyc-green-taxi-trip-explore', tags={}>

In [18]:
def read_greentaxi_dataframe(filename):    
    df = pd.read_parquet(filename)

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df.loc[((df.duration >= 1) & (df.duration <= 60))]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


In [19]:
df_train = read_greentaxi_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet')
df_val = read_greentaxi_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet')

In [20]:
len(df_train), len(df_val)

(54373, 51497)

In [21]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [22]:
categorical = ['PU_DO']#'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
target = 'duration'
Y_train = df_train[target].values
Y_val = df_val[target].values

In [74]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_val)

mean_squared_error(Y_val, Y_pred, squared=False)

5.982652114977233

In [78]:
with open('models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [26]:
with mlflow.start_run():
    
    mlflow.set_tag("developer","Sadat Bin Sharfuddin")

    mlflow.log_param("train-data-url", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-url", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet")

    alpha = 0.001

    mlflow.log_param("alpha",alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, Y_train)

    Y_pred = lr.predict(X_val)

    rmse = mean_squared_error(Y_val, Y_pred, squared=False)

    mlflow.log_metric("rmse",rmse)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [76]:
lr = Ridge(alpha=0.001)
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_val)

mean_squared_error(Y_val, Y_pred, squared=False)

8.036870485972175